In [ ]:
# pandas 사용 선언
# 미리 설치되어 있어야 합니다. conda install pandas 또는 pip install pandas
import pandas as pd

In [ ]:
# CSV 파일을 읽어옵니다.
gfp_df = pd.read_csv("GlobalFirePower.csv")
gfp_df.describe()

In [ ]:
# 데이터프레임의 처음 5개 행을 출력합니다. 데이터의 구조를 파악하기 위해 사용합니다.
gfp_df.head()

In [ ]:
# 데이터의 각 컬럼의 정보를 출력합니다.
gfp_df.info()

In [ ]:
# 데이터프레임을 다루는 간단한 예쩨
ISO3 = gfp_df['ISO3']
country_and_code = gfp_df[['Country', 'ISO3']]
gfp_df[0:3]
gfp_df.loc[0:2]
gfp_df.iloc[0:3]
gfp_df[gfp_df['Active Personnel'] > 500000][['Country', 'ISO3', 'Active Personnel']]

In [ ]:
# 결측치가 있는지 확인해 볼까요?
gfp_df.isnull()

In [ ]:
# 결측치가 있는 행만 출력하려면
gfp_df[gfp_df.isnull().any(axis=1)]

In [ ]:
# 결측치가 있는 컬럼을 확인하려면
missing_in_columns = gfp_df.isnull().sum()
print(missing_in_columns[missing_in_columns > 0])

In [ ]:
# 함수로 만들어서 한 번 사용해 볼까요? (이후 셀에서 재사용 가능))
def report_missing_data(df):
    missing_in_columns = df.isnull().sum()
    print("--- Columns with missing value ---")
    print(missing_in_columns[missing_in_columns > 0])

    missing_in_rows = df[df.isnull().any(axis = 1)]
    print("--- Rows with missing value ---")
    print(missing_in_rows)

report_missing_data(gfp_df)

In [ ]:
# 리마인드
gfp_df.info()

In [ ]:
# 숫자여야 하는데 숫자가 아닌 경우... (Total Population 컬럼에서...)
checking_df = pd.DataFrame()
checking_df['Total Population'] = pd.to_numeric(gfp_df['Total Population'], errors='coerce')
invalid_rows = checking_df[checking_df['Total Population'].isnull()]
print(invalid_rows)

In [ ]:
# 숫자여야 하는데 숫자가 아닌 경우... (전체에서.)
checking_df = pd.DataFrame()
non_numeric_value_location = []
for col in gfp_df.columns:
    if col == 'Country' or col == 'ISO3':
        continue
    for idx, value in gfp_df[col].items():
        if isinstance(value, (int, float)):
            break
        try:
            int(value)
            float(value)
        except ValueError:
            non_numeric_value_location.append((col, idx))

print(len(non_numeric_value_location))
print(non_numeric_value_location)

In [ ]:
# 데이터를 살펴본 후... 
#       Costline (km)의 Landlocked
#       Waterways (km)의 Minimum not met.
#       이 녀석들을 뭘로 정제할 것인가의 이슈
#       0 또는 NAN...
#       NAN으로 바꾸기로 한다.

gfp_df['Coastline (km)'] = pd.to_numeric(gfp_df['Coastline (km)'], errors='coerce')
gfp_df['Waterways (km)'] = pd.to_numeric(gfp_df['Waterways (km)'], errors='coerce')

print(gfp_df['Coastline (km)'])
print(gfp_df['Waterways (km)'])



In [ ]:
# 이제 남은거 두개. 두 개는 손으로 일일이 바꿔주는게 낫다.

checking_df = pd.DataFrame()
non_numeric_value_location = []
for col in gfp_df.columns:
    if col == 'Country' or col == 'ISO3':
        continue
    for idx, value in gfp_df[col].items():
        if isinstance(value, (int, float)):
            break
        try:
            int(value)
            float(value)
        except ValueError:
            non_numeric_value_location.append((col, idx))

for col, idx in non_numeric_value_location:
    print(idx, col, gfp_df.loc[idx, col])

In [ ]:
label = "Total Population"
gfp_df.loc[14, label] = 8174527
gfp_df.at[16, label] = 205823665

for col, idx in non_numeric_value_location:
    print(idx, col, gfp_df.loc[idx, col])

In [ ]:
# 그 다음 결측치. 우리는 처음에 Non-Null Count에서 Active Personnel과 
# Total Aircraft Strength 에 값이 하나씩 빠져 있는 것을 확인했다.

# 결측치를 어떻게 할 것인가?
# 일반적인 방법
    # 그 데이터를 무시한다. 또는 해당 행과 열을 삭제한다.
    # 특정한 값으로 대체한다.
        # 평균값으로 대체 (mean)
        # 중앙값으로 대체 (median)
        # 최빈값으로 대체 (mode)
    # 추정값으로 대체
    # 보간법
# 결측치 자체를 하나의 값으로 취급
# 결측치를 예측 변수로 지정

# 평균값으로 대체
temp_df = gfp_df.copy()     # copy를 해야 함. 파이썬 객체니까. DF에서 copy 메서드 제공
label = 'Active Personnel'
idx = 43
print(temp_df.loc[idx, label])
temp_df.fillna({label: temp_df[label].mean()}, inplace=True)
print(temp_df.loc[idx, label])


In [ ]:
# 중앙값으로 대체
temp_df = gfp_df.copy()     
label = 'Active Personnel'
idx = 43
print(temp_df.loc[idx, label])
temp_df.fillna({label: temp_df[label].median()}, inplace=True)
print(temp_df.loc[idx, label])


In [ ]:
# 중앙값으로 대체
temp_df = gfp_df.copy()     
label = 'Active Personnel'
idx = 43
print(temp_df.loc[idx, label])
temp_df.fillna({label: temp_df[label].mode()[0]}, inplace=True)
print(temp_df.loc[idx, label])


In [ ]:
# 하지만, 최후의 방법은 직접 값을 확인하고 수정하는 것. 가능하다면 이게 가장 좋다.
# 생략된 시리아의 Active Personnel의 값은 154000이다.
print(gfp_df.loc[idx, label])
gfp_df.loc[idx, label] = 154000
print(gfp_df.loc[idx, label])


In [ ]:
# Active Perssonnel / Total Population으로 AP Ratio라는 컬럼을 추가해보자.

temp_df = gfp_df.copy()
temp_df['Total Population'] = temp_df['Total Population'].astype('int64')


temp_df['AP Ratio'] = temp_df['Active Personnel'] / temp_df['Total Population']
temp_df.info()


In [ ]:
temp_df['AP Ratio']

In [ ]:
# 데이터 정렬
pd.set_option('display.max_rows', None)  # Set the maximum number of rows to display as None
temp_df = temp_df.sort_values(by='AP Ratio', ascending=False)
temp_df[['Country', 'AP Ratio']]


In [ ]:
# 1등부터 100등까지 줄을 세운 후
# 25등의 비율, 75등의 비율을 기준으로 그 위, 그 아래가 비정상이라고 판단하면 어떨까?
percentile_75, percentile_25 = temp_df['AP Ratio'].quantile(0.75), temp_df['AP Ratio'].quantile(0.25)
print(percentile_75, percentile_25)
temp_df[temp_df['AP Ratio'] > percentile_75]

In [ ]:
temp_df[temp_df['AP Ratio'] < percentile_25]

In [ ]:
# AP Ratio 컬럼을 기준으로, X 축에는 0에서 1까지의 값을 가지고
# 각각의 나라의 점을 찍고 싶은데, 점 위에는 ISO3 레이블이 표시되도록 그림을 matplotlib으로 그릴 수 있을까?

import matplotlib.pyplot as plt

# Set the x-axis range from 0 to 1
plt.xlim(0, 1)

# Plot the data points
plt.scatter(temp_df['AP Ratio'], temp_df['ISO3'])

# Add labels to each point
for i, iso3 in enumerate(temp_df['ISO3']):
    plt.annotate(iso3, (temp_df['AP Ratio'].iloc[i], temp_df['ISO3'].iloc[i]), textcoords="offset points", xytext=(0,10), ha='center')

# plt.annotate는 각 점에 레이블을 붙이는 함수로 plt.text와 비슷하지만, 좀 더 세밀한 조정이 가능하다.
    
# Set the x-axis label
plt.xlabel('AP Ratio')

# Set the title
plt.title('AP Ratio by Country')

# Show the plot
plt.show()

# 존 튜키의 제안 - 75 퍼센타일 값의 1.5배를 넘거나, 25 퍼센타일 값의 1.5배 이하면 



In [ ]:
upper_bound = temp_df['AP Ratio'].max()
lower_bound = temp_df['AP Ratio'].min()


# Set the x-axis range from 0 to 1
plt.figure(figsize=(50, 6))

plt.xlim(lower_bound, upper_bound)
# Plot the data points
plt.scatter(temp_df['AP Ratio'], temp_df['ISO3'])

# Add labels to each point
for i, iso3 in enumerate(temp_df['ISO3']):
    plt.annotate(iso3, (temp_df['AP Ratio'].iloc[i], temp_df['ISO3'].iloc[i]), textcoords="offset points", xytext=(0,10), ha='center')

# Set the x-axis label
plt.xlabel('AP Ratio')

# Set the title
plt.title('AP Ratio by Country')

# Show the plot
plt.show()

# 존 튜키의 제안 - 75 퍼센타일 값의 1.5배를 넘거나, 25 퍼센타일 값의 1.5배 이하면 

In [ ]:
# Set the x-axis range from 0 to 1
plt.figure(figsize=(50, 6))

plt.xlim(lower_bound, upper_bound)


# percentile_75 초과 또는 percentile_25 미만은 빨간색, 나머지는 파란색으로 표시
# Plot the data points with color based on AP Ratio values
plt.scatter(temp_df['AP Ratio'], temp_df['ISO3'], c=['red' if x > percentile_75 or x < percentile_25 else 'blue' for x in temp_df['AP Ratio']], cmap='coolwarm')
#plt.scatter(temp_df['AP Ratio'], temp_df['ISO3'], c=temp_df['AP Ratio'], cmap='coolwarm')

# Add labels to each point
for i, iso3 in enumerate(temp_df['ISO3']):
    plt.annotate(iso3, (temp_df['AP Ratio'].iloc[i], temp_df['ISO3'].iloc[i]), textcoords="offset points", xytext=(0,10), ha='center')

# Set the x-axis label
plt.xlabel('AP Ratio')

# Set the title
plt.title('AP Ratio by Country')

# Show the colorbar
plt.colorbar()

# Show the plot
plt.show()


In [ ]:
# 너무 가혹하다. 낮은 국가들은 비슷비슷한데...
IQR = percentile_75 - percentile_25
upper_outlier = percentile_75 + (IQR * 1.5)
lower_outlier = percentile_25 - (IQR * 1.5)

print(upper_outlier, lower_outlier)
# Set the x-axis range from 0 to 1
plt.figure(figsize=(50, 6))

plt.xlim(lower_bound, upper_bound)


# percentile_75 초과 또는 percentile_25 미만은 빨간색, 나머지는 파란색으로 표시
# Plot the data points with color based on AP Ratio values
plt.scatter(temp_df['AP Ratio'], temp_df['ISO3'], c=['red' if x > upper_outlier or x < lower_outlier else 'blue' for x in temp_df['AP Ratio']], cmap='coolwarm')
#plt.scatter(temp_df['AP Ratio'], temp_df['ISO3'], c=temp_df['AP Ratio'], cmap='coolwarm')

# Add labels to each point
for i, iso3 in enumerate(temp_df['ISO3']):
    plt.annotate(iso3, (temp_df['AP Ratio'].iloc[i], temp_df['ISO3'].iloc[i]), textcoords="offset points", xytext=(0,10), ha='center')

# Set the x-axis label
plt.xlabel('AP Ratio')

# Set the title
plt.title('AP Ratio by Country')

# Show the colorbar
plt.colorbar()

# Show the plot
plt.show()


# IQR의 문제점


In [ ]:
# Z 점수 : 어떤 데이터가 평균으로부터 얼마나 떨어져 있는지를 검사하는 통계적 지표
# Z = (X - μ) / σ , X는 개별 데이터, μ는 평균, σ는 표준편차
# Z 점수가 -1 ~ 1인 데이터 : 통계적으로 68%의 데이터가 이 범위에 속함
# Z 점수가 -2 ~ 2인 데이터 : 통계적으로 95%의 데이터가 이 범위에 속함
# Z 점수가 -3 ~ 3인 데이터 : 통계적으로 99.7%의 데이터가 이 범위에 속함

# AP Ratio의 평균값 
mean = temp_df['AP Ratio'].mean()
# AP Ratio의 표준편차
std = temp_df['AP Ratio'].std()

# 다음 함수는 개별 데이터의 Z 점수를 반환함
def z_score(x, mean, std):
    return (x - mean) / std

# Z 점수가 -2 미만, 2 초과인 데이터를 이상치로 판단하고 빨간색으로 표시해서 그래프를 그림
plt.figure(figsize=(50, 6))
plt.xlim(lower_bound, upper_bound)
plt.scatter(temp_df['AP Ratio'], temp_df['ISO3'], c=['red' if abs(z_score(x, mean, std)) > 2 else 'blue' for x in temp_df['AP Ratio']], cmap='coolwarm')
for i, iso3 in enumerate(temp_df['ISO3']):
    plt.annotate(iso3, (temp_df['AP Ratio'].iloc[i], temp_df['ISO3'].iloc[i]), textcoords="offset points", xytext=(0,10), ha='center')
plt.xlabel('AP Ratio')
plt.title('AP Ratio by Country')
plt.show()


- 인구대비 지나치게 병력이 많은 이상국가는 있지만, 인구대비 지나치게 병력이 작은 이상 국가는 찾을 수 없다.
- 무엇이 문제일까?
- 일단 이 데이터가 Z 분포로 조사가 가능한 데이터인지를 확인해보자. 정규분포 여부를 조사해보자.


In [ ]:
# AP Ratio가 정규 분포를 따르는지를 확인
import seaborn as sns
sns.histplot(temp_df['AP Ratio'], kde=True)
plt.show()


- 일반적으로 말하는 정규분포와 거리가 있다.


In [ ]:
# 만약 NKO를 제외하면 어떨까? NKO를 제외하고 정규분포를 따르는지 확인해보자.
temp_df_without_nko = temp_df[temp_df['ISO3'] != 'NKO']
sns.histplot(temp_df_without_nko['AP Ratio'], kde=True)
plt.show()


In [ ]:
import numpy as np
# 역시 정규분포와 거리가 멀다. 이런 경우에는 로그 변환을 해보는 것도 방법이다.
temp_df['AP Ratio LOG'] = temp_df['AP Ratio'].apply(lambda x: np.log(x))
sns.histplot(temp_df['AP Ratio LOG'], kde=True)
plt.show()


In [ ]:
outlier_level = 2
# 이제 이 LOG값을 기준으로 이상치를 찾아보자.
mean = temp_df['AP Ratio LOG'].mean()
std = temp_df['AP Ratio LOG'].std()
temp_df['Z Score'] = temp_df['AP Ratio LOG'].apply(lambda x: z_score(x, mean, std))

plt.figure(figsize=(50, 6))
plt.xlim(temp_df['AP Ratio LOG'].min(), temp_df['AP Ratio LOG'].max())
plt.scatter(temp_df['AP Ratio LOG'], temp_df['ISO3'], c=['red' if abs(x) > outlier_level else 'blue' for x in temp_df['Z Score']], cmap='coolwarm')

# plt.scatter(temp_df['AP Ratio LOG'], temp_df['ISO3'], c=['red' if abs(z_score(x, mean, std)) > outlier_lever else 'blue' for x in temp_df['AP Ratio LOG']], cmap='coolwarm')
for i, iso3 in enumerate(temp_df['ISO3']):
    plt.annotate(iso3, (temp_df['AP Ratio LOG'].iloc[i], temp_df['ISO3'].iloc[i]), textcoords="offset points", xytext=(0,10), ha='center')
plt.xlabel('AP Ratio LOG')
plt.title('AP Ratio by Country')
plt.show()

# 이상치에 해당하는 값의 Country, ISO3, AP_RATIO값과 Z 점수를 출력
print(f"평균: {mean}, 표준편차: {std}")
outliers = temp_df[np.abs(temp_df['Z Score']) > outlier_level]
print(outliers[['Country', 'ISO3', 'AP Ratio', 'AP Ratio LOG', 'Z Score']].sort_values(by='AP Ratio LOG'))




In [ ]:
# AP RATIO LOG가 정말 정규 분포를 따르는지 통계적으로 검증해보자.
from scipy.stats import shapiro
shapiro_test = shapiro(temp_df['AP Ratio LOG'])
print(shapiro_test) 

- shaipro 함수는 두 개의 값을 반환한다. 첫 번째 값은 통계량, 두 번째 값은 p-value.
- 통계량은 이 데이터가 얼마나 정규분포를 따르는지를 계산한 것이며
- p-value는 통계량을 기준으로 귀무가설 검정을 위한 유의확률을 계산한 것이다.

- 가설 검정에 대하여
- 귀무가설 : AP Ratio LOG는 정규분포를 따른다.
- 대립가설 : AP Ratio LOG는 정규분포를 따르지 않는다.
- p-value < 0.05 : 귀무가설을 기각 (95% 유의수준)
- p-value >= 0.05 : 대립가설을 기각
- 테스트 결과는 p value가 0.2857 이므로 95% 신뢰수준에서 정규분포를 따른다고 볼 수 있다.


In [ ]:
# 위의 분석을 NKO를 제외하고 다시 해 보자.
temp_df_without_nko['AP Ratio LOG'] = temp_df_without_nko['AP Ratio'].apply(lambda x: np.log(x))
sns.histplot(temp_df_without_nko['AP Ratio LOG'], kde=True)
plt.show()


In [ ]:
# 평균과 표준편차는 NKO를 제외하고 계산하지만, 이상치를 찾을 때는 NKO를 포함해서 찾는다.
mean_without_nko = temp_df_without_nko['AP Ratio LOG'].mean()
std_without_nko = temp_df_without_nko['AP Ratio LOG'].std()


temp_df['Z Score WO NKO'] = temp_df['AP Ratio LOG'].apply(lambda x: z_score(x, mean_without_nko, std_without_nko))

plt.figure(figsize=(50, 6))
plt.xlim(temp_df['AP Ratio LOG'].min(), temp_df['AP Ratio LOG'].max())
plt.scatter(temp_df['AP Ratio LOG'], temp_df['ISO3'], c=['red' if abs(x) > outlier_level else 'blue' for x in temp_df['Z Score WO NKO']], cmap='coolwarm')

# plt.scatter(temp_df['AP Ratio LOG'], temp_df['ISO3'], c=['red' if abs(z_score(x, mean, std)) > outlier_lever else 'blue' for x in temp_df['AP Ratio LOG']], cmap='coolwarm')
for i, iso3 in enumerate(temp_df['ISO3']):
    plt.annotate(iso3, (temp_df['AP Ratio LOG'].iloc[i], temp_df['ISO3'].iloc[i]), textcoords="offset points", xytext=(0,10), ha='center')
plt.xlabel('AP Ratio LOG')
plt.title('AP Ratio by Country')
plt.show()

# 이상치에 해당하는 값의 Country, ISO3, AP_RATIO값과 Z 점수를 출력
print(f"평균: {mean_without_nko}, 표준편차: {std_without_nko}")
outliers = temp_df[np.abs(temp_df['Z Score WO NKO']) > outlier_level]
print(outliers[['Country', 'ISO3', 'AP Ratio', 'AP Ratio LOG', 'Z Score WO NKO']].sort_values(by='AP Ratio LOG'))




- NKO라는 특이점을 배제하고 분석한 결과 TAN이 이상치에 포함되었다.
- 제외하는 것이 옳은지의 판단은 데이터 과학자의 감이다. 이를 합리적으로 설명할 수 있어야 한다.


# 상관관계 분석

- 두 번째 분석. 해군력에 대해서 분석해보자.


In [ ]:
# gfp_df에서 해군력과 관련된 컬럼만 추출해서 새로운 df를 만든다.
# 추출할 컬럼은 다음과 같다.
#   Country, ISO3, Rank, Total Naval Assets, Aircraft Carriers, Frigates, Destroyers, Corvettes, Submarines, Patrol Craft, Mine Warfare Vessels
navy_df = gfp_df[['Country', 'ISO3', 'Rank', 'Total Naval Assets', 'Aircraft Carriers', 'Frigates', 'Destroyers', 'Corvettes', 'Submarines', 'Patrol Craft', 'Mine Warfare Vessels']]
navy_df.info()

In [ ]:
# navy_df의 각 변수의 상관관계를 구해보자.
# 단 상관관계는 숫자가 아닌 데이터에서 구할 수 없으므로 Country, ISO3는 제외한다.
navy_df_corr = navy_df.drop(['Country', 'ISO3'], axis=1).corr()
navy_df_corr


In [ ]:
# 상관관계를 시각화해보자.
sns.heatmap(navy_df_corr, annot=True, cmap='coolwarm')
plt.show()


In [ ]:
# 모두 양의 상관관계를 가지고 있고, 어떤 컬럼은 매우 높은, 어떤 컬럼은 매우 낮은 상관관계를 보인다.
# 그렇다면 total navel assets을 RANK, ISO3, Country를 제외한 나머지 컬럼으로 예측할 수 있을까?
# statsmodels 라이브러리를 사용해서 회귀분석을 해보자.
import statsmodels.api as sm
dependent_variable = 'Total Naval Assets'
columns = ['Aircraft Carriers', 'Frigates', 'Destroyers', 'Corvettes', 'Submarines', 'Patrol Craft', 'Mine Warfare Vessels']
X = navy_df[columns]
X = sm.add_constant(X)
y = navy_df[dependent_variable]

model = sm.OLS(y, X).fit()
model.summary()

In [ ]:
# R-squared 값이 0.905인데, 이는 90.5%의 설명력을 가진다는 것을 의미한다.
# 이 모델은 설명력이 매우 높다고 볼 수 있다.

# 이제 이 모델을 사용해서 모든 국가의 Total Naval Assets을 예측해보자.
# 예측값과 실제값을 비교해보자.
navy_df['Predicted Total Naval Assets'] = model.predict(X)
navy_df[['Country', 'Total Naval Assets', 'Predicted Total Naval Assets']]
navy_df['Difference'] = navy_df['Total Naval Assets'] - navy_df['Predicted Total Naval Assets']
navy_df[['Country', 'Total Naval Assets', 'Predicted Total Naval Assets', 'Difference']]
for c, t, p, d in navy_df[['Country', 'Total Naval Assets', 'Predicted Total Naval Assets', 'Difference']].values:
    print(f"{c} - {t} - {p} - {d}")
print(navy_df[['Country', 'Total Naval Assets', 'Predicted Total Naval Assets', 'Difference']])



In [ ]:
# 잘 모르겠으니 각 국가별로 차이를 그래프로 그려보자.
plt.figure(figsize=(50, 6))
plt.bar(navy_df['Country'], navy_df['Difference'])
plt.xticks(rotation=90)
plt.show()


- 단순한 회귀 분석으로 분석 결과를 얻을 수 없다는 것은 자명하다.
- 분석 측에서는 공개하지 않은 추가적인 고려 사항이 있다.
